## Comparing Diverse Cities to help People Relocate

## 1. Download Toronto City into Dataframe TO_neighborhoods

In [37]:
import pandas as pd # library for data analsysis
import requests # library to handle requests


In [38]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
htmltext = url.text
print('XML Downloaded')

XML Downloaded


In [39]:
from bs4 import BeautifulSoup
import bs4

soup = bs4.BeautifulSoup(htmltext, 'lxml')
table = soup.find('table','wikitable sortable')
#print (table)

In [40]:
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text().replace('\n', ''))
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
            print (column_names)
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
df = df.replace('\n','', regex=True)
df = df[df.Borough != "Not assigned"]
df.head()

['Postcode', 'Borough', 'Neighbourhood']


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [41]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
    
    

--2019-03-30 13:19:53--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-03-30 13:19:53--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-30 13:19:54--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18r

In [42]:
Geospatial_data_df = pd.read_csv("Geospatial_data.csv")

Geospatial_data_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
TO_neighborhoods = pd.merge(df, Geospatial_data_df, on='Postcode')

TO_neighborhoods.rename(columns={'Postcode': 'City'}, inplace=True)
TO_neighborhoods.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
TO_neighborhoods['City'] = 'TO'
TO_neighborhoods.head()

downtown_toronto_data = TO_neighborhoods[TO_neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto_data['Borough'] = 'TO_' + downtown_toronto_data['Borough']
downtown_toronto_data['Neighborhood'] = 'TO_' + downtown_toronto_data['Neighborhood'] 
downtown_toronto_data.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,TO,TO_Downtown Toronto,TO_Harbourfront,43.654260,-79.360636
1,TO,TO_Downtown Toronto,TO_Regent Park,43.654260,-79.360636
2,TO,TO_Downtown Toronto,TO_Ryerson,43.657162,-79.378937
3,TO,TO_Downtown Toronto,TO_Garden District,43.657162,-79.378937
4,TO,TO_Downtown Toronto,TO_St. James Town,43.651494,-79.375418


<a id='item1'></a>

## 1. Download New York City into Dataframe NY_neighborhoods

In [43]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [44]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as NY_json_data:
    NY_data = json.load(NY_json_data)
    
# Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. 
# So, let's define a new variable that includes this data   

NY_features_data = NY_data['features']

# Tranform the JSON data into a pandas dataframe
# define the dataframe columns
column_names = ['City', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
NY_neighborhoods = pd.DataFrame(columns=column_names)

for data in NY_features_data:
    city = 'NY'
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NY_neighborhoods = NY_neighborhoods.append({'City': city,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

print ("NY Loaded")

NY Loaded


In [45]:
NY_neighborhoods.head()
NY_neighborhoods.shape

(306, 5)

# need to just load manhattan since you can only make so many requests to Foursqare API

In [46]:
manhattan_data = NY_neighborhoods[NY_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

In [47]:
manhattan_data['Borough'] = 'NY_' + manhattan_data['Borough'] 
manhattan_data['Neighborhood'] = 'NY_' + manhattan_data['Neighborhood'] 
manhattan_data.head()


,City,Borough,Neighborhood,Latitude,Longitude
0,NY,NY_Manhattan,NY_Marble Hill,40.876551,-73.910660
1,NY,NY_Manhattan,NY_Chinatown,40.715618,-73.994279
2,NY,NY_Manhattan,NY_Washington Heights,40.851903,-73.936900
3,NY,NY_Manhattan,NY_Inwood,40.867684,-73.921210
4,NY,NY_Manhattan,NY_Hamilton Heights,40.823604,-73.949688


## 1. Concatenate the NY neighborhoods with Toronto neighborhoods into neighborhoods

In [48]:
TO_neighborhoods.reset_index()
neighborhood_data = pd.concat([manhattan_data, downtown_toronto_data], axis=0).reset_index()
neighborhood_data.head()

,index,City,Borough,Neighborhood,Latitude,Longitude
0,0,NY,NY_Manhattan,NY_Marble Hill,40.876551,-73.910660
1,1,NY,NY_Manhattan,NY_Chinatown,40.715618,-73.994279
2,2,NY,NY_Manhattan,NY_Washington Heights,40.851903,-73.936900
3,3,NY,NY_Manhattan,NY_Inwood,40.867684,-73.921210
4,4,NY,NY_Manhattan,NY_Hamilton Heights,40.823604,-73.949688


In [49]:
neighborhood_data.tail()

,index,City,Borough,Neighborhood,Latitude,Longitude
72,32,TO,TO_Downtown Toronto,TO_Cabbagetown,43.667967,-79.367675
73,33,TO,TO_Downtown Toronto,TO_St. James Town,43.667967,-79.367675
74,34,TO,TO_Downtown Toronto,TO_First Canadian Place,43.648429,-79.382280
75,35,TO,TO_Downtown Toronto,TO_Underground city,43.648429,-79.382280
76,36,TO,TO_Downtown Toronto,TO_Church and Wellesley,43.665860,-79.383160


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [50]:
CLIENT_ID = 'JVFWKWHDS0TWEDRK4F44GHOHWOS1DOHFZLRI14J1BKB5YO12' # your Foursquare ID
CLIENT_SECRET = 'HFR4PME5VUMUESCRPCDIGP2OBHC2HI4TF4ZYHEFZM3NA0FHP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JVFWKWHDS0TWEDRK4F44GHOHWOS1DOHFZLRI14J1BKB5YO12
CLIENT_SECRET:HFR4PME5VUMUESCRPCDIGP2OBHC2HI4TF4ZYHEFZM3NA0FHP


Now we are ready to clean the json and structure it into a *pandas* dataframe.

<a id='item2'></a>

In [51]:
## Explore the neighborhoods or NY and TO

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [53]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

neighborhood_venues = getNearbyVenues(names=neighborhood_data['Neighborhood'],
                                   latitudes=neighborhood_data['Latitude'],
                                   longitudes=neighborhood_data['Longitude']
                                  )



NY_Marble Hill
NY_Chinatown
NY_Washington Heights
NY_Inwood
NY_Hamilton Heights
NY_Manhattanville
NY_Central Harlem
NY_East Harlem
NY_Upper East Side
NY_Yorkville
NY_Lenox Hill
NY_Roosevelt Island
NY_Upper West Side
NY_Lincoln Square
NY_Clinton
NY_Midtown
NY_Murray Hill
NY_Chelsea
NY_Greenwich Village
NY_East Village
NY_Lower East Side
NY_Tribeca
NY_Little Italy
NY_Soho
NY_West Village
NY_Manhattan Valley
NY_Morningside Heights
NY_Gramercy
NY_Battery Park City
NY_Financial District
NY_Carnegie Hill
NY_Noho
NY_Civic Center
NY_Midtown South
NY_Sutton Place
NY_Turtle Bay
NY_Tudor City
NY_Stuyvesant Town
NY_Flatiron
NY_Hudson Yards
TO_Harbourfront
TO_Regent Park
TO_Ryerson
TO_Garden District
TO_St. James Town
TO_Berczy Park
TO_Central Bay Street
TO_Christie
TO_Adelaide
TO_King
TO_Richmond
TO_Harbourfront East
TO_Toronto Islands
TO_Union Station
TO_Design Exchange
TO_Toronto Dominion Centre
TO_Commerce Court
TO_Victoria Hotel
TO_Harbord
TO_University of Toronto
TO_Chinatown
TO_Grange Park
T

#### Let's check the size of the resulting dataframe

In [54]:
print(neighborhood_venues.shape)
neighborhood_venues.head(10)

(5815, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,NY_Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,NY_Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,NY_Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,NY_Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.877136,-73.906666,Donut Shop
4,NY_Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
5,NY_Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
6,NY_Marble Hill,40.876551,-73.91066,Blink Fitness Riverdale,40.877147,-73.905837,Gym
7,NY_Marble Hill,40.876551,-73.91066,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium
8,NY_Marble Hill,40.876551,-73.91066,Starbucks,40.873755,-73.908613,Coffee Shop
9,NY_Marble Hill,40.876551,-73.91066,T.J. Maxx,40.877232,-73.905042,Department Store


Let's check how many venues were returned for each neighborhood

In [55]:
neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
NY_Battery Park City,100,100,100,100,100,100
NY_Carnegie Hill,100,100,100,100,100,100
NY_Central Harlem,45,45,45,45,45,45
NY_Chelsea,100,100,100,100,100,100
NY_Chinatown,100,100,100,100,100,100
NY_Civic Center,100,100,100,100,100,100
NY_Clinton,100,100,100,100,100,100
NY_East Harlem,47,47,47,47,47,47
NY_East Village,100,100,100,100,100,100


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [56]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [57]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,NY_Battery Park City,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.00,0.00,0.010000,0.030000,0.000000,0.010000
1,NY_Carnegie Hill,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.0,0.00,0.000000,0.00,0.00,0.010000,0.030000,0.000000,0.010000
2,NY_Central Harlem,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
3,NY_Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.010000
4,NY_Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.030000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
5,NY_Civic Center,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.00,0.00,0.010000,0.020000,0.010000,0.000000
6,NY_Clinton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.00,0.00,0.020000,0.030000,0.000000,0.000000
7,NY_East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
8,NY_East Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.0,0.00,0.000000,0.00,0.00,0.050000,0.020000,0.000000,0.000000
9,NY_Financial District,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.00,0.00,0.010000,0.040000,0.000000,0.010000


#### Let's print each neighborhood along with the top 5 most common venues

In [58]:
num_top_venues = 5

for hood in neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhood_grouped[neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----NY_Battery Park City----
         venue  freq
0         Park  0.08
1  Coffee Shop  0.07
2        Hotel  0.05
3          Gym  0.04
4    Wine Shop  0.03


----NY_Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1            Café  0.05
2     Coffee Shop  0.05
3  Cosmetics Shop  0.04
4     Yoga Studio  0.03


----NY_Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1        Cosmetics Shop  0.04
2    Seafood Restaurant  0.04
3    Chinese Restaurant  0.04
4  Gym / Fitness Center  0.04


----NY_Chelsea----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2      Ice Cream Shop  0.05
3           Nightclub  0.04
4              Bakery  0.04


----NY_Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1         Cocktail Bar  0.04
2   Dim Sum Restaurant  0.04
3      Bubble Tea Shop  0.04
4  American Restaurant  0.04


----NY_Civic Center----
                  venue  freq
0  Gym / Fitness Center 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [59]:
import numpy as np # library to handle data in a vectorized manner

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NY_Battery Park City,Park,Coffee Shop,Hotel,Gym,Wine Shop,Food Truck,Italian Restaurant,Plaza,Playground,BBQ Joint
1,NY_Carnegie Hill,Pizza Place,Coffee Shop,Café,Cosmetics Shop,Yoga Studio,Bookstore,Japanese Restaurant,Gym,Spa,French Restaurant
2,NY_Central Harlem,African Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,American Restaurant,Cosmetics Shop,French Restaurant,Public Art,Café,Salon / Barbershop
3,NY_Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Seafood Restaurant,Hotel,Theater,Market
4,NY_Chinatown,Chinese Restaurant,American Restaurant,Cocktail Bar,Dim Sum Restaurant,Bubble Tea Shop,Hotpot Restaurant,Ice Cream Shop,Bakery,Vietnamese Restaurant,Salon / Barbershop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [61]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 8

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 0, 0, 0, 7, 0, 6, 7, 0, 6], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [62]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhood_merged = neighborhood_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhood_merged = neighborhood_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhood_merged.head() # check the last columns!

,index,City,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,NY,NY_Manhattan,NY_Marble Hill,40.876551,-73.910660,6,Discount Store,Coffee Shop,Yoga Studio,Gym,Steakhouse,Big Box Store,Supplement Shop,Shoe Store,Donut Shop,Bank
1,1,NY,NY_Manhattan,NY_Chinatown,40.715618,-73.994279,7,Chinese Restaurant,American Restaurant,Cocktail Bar,Dim Sum Restaurant,Bubble Tea Shop,Hotpot Restaurant,Ice Cream Shop,Bakery,Vietnamese Restaurant,Salon / Barbershop
2,2,NY,NY_Manhattan,NY_Washington Heights,40.851903,-73.936900,7,Café,Bakery,Spanish Restaurant,Grocery Store,Mobile Phone Shop,Gym,Sandwich Place,Clothing Store,Wine Shop,Tapas Restaurant
3,3,NY,NY_Manhattan,NY_Inwood,40.867684,-73.921210,7,Mexican Restaurant,Café,Pizza Place,Lounge,Chinese Restaurant,Restaurant,Bakery,Caribbean Restaurant,Frozen Yogurt Shop,Park
4,4,NY,NY_Manhattan,NY_Hamilton Heights,40.823604,-73.949688,7,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Deli / Bodega,Sushi Restaurant,School,Sandwich Place,Caribbean Restaurant,Chinese Restaurant


Finally, let's visualize the resulting clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [63]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 0, neighborhood_merged.columns[[3] + list(range(5, neighborhood_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,NY_Manhattanville,-73.957385,0,Deli / Bodega,Coffee Shop,Seafood Restaurant,Italian Restaurant,Park,Mexican Restaurant,Dumpling Restaurant,Asian Restaurant,Juice Bar,Bike Trail
6,NY_Central Harlem,-73.943211,0,African Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,American Restaurant,Cosmetics Shop,French Restaurant,Public Art,Café,Salon / Barbershop
8,NY_Upper East Side,-73.960508,0,Italian Restaurant,Exhibit,Coffee Shop,Art Gallery,Bakery,Juice Bar,Gym / Fitness Center,French Restaurant,Boutique,Hotel
9,NY_Yorkville,-73.947118,0,Italian Restaurant,Bar,Gym,Coffee Shop,Pizza Place,Diner,Mexican Restaurant,Japanese Restaurant,Deli / Bodega,Sushi Restaurant
10,NY_Lenox Hill,-73.958860,0,Sushi Restaurant,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pizza Place,Sporting Goods Shop,Burger Joint,Gym,Deli / Bodega,Cosmetics Shop
12,NY_Upper West Side,-73.977059,0,Italian Restaurant,Bar,Coffee Shop,Indian Restaurant,Bakery,Vegetarian / Vegan Restaurant,Cosmetics Shop,Wine Bar,Bookstore,Pub
13,NY_Lincoln Square,-73.985338,0,Theater,Gym / Fitness Center,Italian Restaurant,Plaza,Café,Concert Hall,French Restaurant,Opera House,Park,Indie Movie Theater
17,NY_Chelsea,-74.003116,0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Seafood Restaurant,Hotel,Theater,Market
18,NY_Greenwich Village,-73.999914,0,Italian Restaurant,Sushi Restaurant,French Restaurant,Clothing Store,Boutique,Bakery,Chinese Restaurant,Indian Restaurant,Café,Seafood Restaurant
19,NY_East Village,-73.982226,0,Bar,Wine Bar,Ice Cream Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Pizza Place,Ramen Restaurant,Cocktail Bar,Chinese Restaurant


#### Cluster 2

In [64]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 1, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,City,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,TO,-79.377529,1,Park,Trail,Playground,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner


#### Cluster 3

In [65]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 2, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,City,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,TO,-79.39442,2,Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane
64,TO,-79.39442,2,Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane
65,TO,-79.39442,2,Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane
66,TO,-79.39442,2,Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane
67,TO,-79.39442,2,Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane
68,TO,-79.39442,2,Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane
69,TO,-79.39442,2,Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane


#### Cluster 4

In [66]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 3, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,City,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,NY,-73.963896,3,Coffee Shop,Bookstore,Park,American Restaurant,Sandwich Place,Deli / Bodega,Food Truck,Burger Joint,Tennis Court,Outdoor Sculpture
40,TO,-79.360636,3,Coffee Shop,Café,Pub,Park,Bakery,Restaurant,Breakfast Spot,Theater,Mexican Restaurant,Shoe Store
41,TO,-79.360636,3,Coffee Shop,Café,Pub,Park,Bakery,Restaurant,Breakfast Spot,Theater,Mexican Restaurant,Shoe Store
44,TO,-79.375418,3,Coffee Shop,Restaurant,Café,Hotel,Bakery,Italian Restaurant,Park,Breakfast Spot,Clothing Store,Gastropub
45,TO,-79.373306,3,Coffee Shop,Cocktail Bar,Pub,Restaurant,Steakhouse,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Bakery
46,TO,-79.387383,3,Coffee Shop,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Café,Bar,Spa,Japanese Restaurant,Sushi Restaurant
51,TO,-79.381752,3,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Scenic Lookout,Brewery,Pizza Place,Fried Chicken Joint,Sports Bar
52,TO,-79.381752,3,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Scenic Lookout,Brewery,Pizza Place,Fried Chicken Joint,Sports Bar
53,TO,-79.381752,3,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Scenic Lookout,Brewery,Pizza Place,Fried Chicken Joint,Sports Bar
54,TO,-79.381576,3,Coffee Shop,Café,Hotel,American Restaurant,Gastropub,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Restaurant,Concert Hall


#### Cluster 5

In [67]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 4, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,City,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,TO,-79.422564,4,Café,Grocery Store,Park,Italian Restaurant,Diner,Athletics & Sports,Nightclub,Coffee Shop,Restaurant,Baby Store


#### Cluster 6

In [68]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 5, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,City,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,NY,-73.974052,5,Bar,Park,Boat or Ferry,Baseball Field,Basketball Court,Cocktail Bar,Coffee Shop,Heliport,Gas Station,Harbor / Marina


#### Cluster 7

In [69]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 6, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,City,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NY,-73.910660,6,Discount Store,Coffee Shop,Yoga Studio,Gym,Steakhouse,Big Box Store,Supplement Shop,Shoe Store,Donut Shop,Bank
11,NY,-73.949168,6,Sandwich Place,Coffee Shop,Japanese Restaurant,Dry Cleaner,Deli / Bodega,Outdoors & Recreation,Bus Line,Scenic Lookout,Park,Metro Station
14,NY,-73.996119,6,Theater,Hotel,Italian Restaurant,Gym / Fitness Center,American Restaurant,Wine Shop,Gym,Spa,Lounge,Mediterranean Restaurant
15,NY,-73.981669,6,Clothing Store,Hotel,Theater,Steakhouse,Cocktail Bar,Coffee Shop,Spa,Food Truck,Bookstore,Bakery
16,NY,-73.978332,6,Hotel,Coffee Shop,American Restaurant,Japanese Restaurant,French Restaurant,Italian Restaurant,Spa,Bar,Sandwich Place,Gym
28,NY,-74.016869,6,Park,Coffee Shop,Hotel,Gym,Wine Shop,Food Truck,Italian Restaurant,Plaza,Playground,BBQ Joint
29,NY,-74.010665,6,Coffee Shop,Steakhouse,Hotel,Wine Shop,Gym,Bar,Italian Restaurant,Pizza Place,Café,Park
33,NY,-73.988713,6,Korean Restaurant,Hotel,Hotel Bar,Cosmetics Shop,Japanese Restaurant,Gym / Fitness Center,Italian Restaurant,Bakery,Cocktail Bar,Boutique
35,NY,-73.967708,6,Sushi Restaurant,Italian Restaurant,Steakhouse,Coffee Shop,Hotel,Japanese Restaurant,French Restaurant,Park,Café,Ramen Restaurant
36,NY,-73.971219,6,Park,Mexican Restaurant,Sushi Restaurant,Café,Diner,Greek Restaurant,Hotel,Deli / Bodega,Dog Run,Pizza Place


#### Cluster 8

In [70]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 7, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,City,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,NY,-73.994279,7,Chinese Restaurant,American Restaurant,Cocktail Bar,Dim Sum Restaurant,Bubble Tea Shop,Hotpot Restaurant,Ice Cream Shop,Bakery,Vietnamese Restaurant,Salon / Barbershop
2,NY,-73.936900,7,Café,Bakery,Spanish Restaurant,Grocery Store,Mobile Phone Shop,Gym,Sandwich Place,Clothing Store,Wine Shop,Tapas Restaurant
3,NY,-73.921210,7,Mexican Restaurant,Café,Pizza Place,Lounge,Chinese Restaurant,Restaurant,Bakery,Caribbean Restaurant,Frozen Yogurt Shop,Park
4,NY,-73.949688,7,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Deli / Bodega,Sushi Restaurant,School,Sandwich Place,Caribbean Restaurant,Chinese Restaurant
7,NY,-73.944182,7,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Spa,Deli / Bodega,Pizza Place,Liquor Store,Sandwich Place,Bank
20,NY,-73.980890,7,Café,Coffee Shop,Art Gallery,Ramen Restaurant,Pizza Place,Cocktail Bar,Chinese Restaurant,Japanese Restaurant,Shoe Store,Bakery
58,TO,-79.400049,7,Café,Bakery,Bar,Japanese Restaurant,Bookstore,Restaurant,Coffee Shop,Beer Bar,Sushi Restaurant,Beer Store
59,TO,-79.400049,7,Café,Bakery,Bar,Japanese Restaurant,Bookstore,Restaurant,Coffee Shop,Beer Bar,Sushi Restaurant,Beer Store
60,TO,-79.400049,7,Café,Bar,Vegetarian / Vegan Restaurant,Bakery,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Dessert Shop,Dim Sum Restaurant
61,TO,-79.400049,7,Café,Bar,Vegetarian / Vegan Restaurant,Bakery,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Dessert Shop,Dim Sum Restaurant
